Import the csv with the plot labels and their coordinates

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Thesis/Chapter2_simulations/locations/SWRuns_InputMain_BSE898.csv')

In [ ]:
site = df['Label']

need to swith from negative to positive coordinates for Livneh

In [ ]:
lon = 360+df['X_WGS84']

In [ ]:
lat = df['Y_WGS84']

For some reason for tmax, 1927 and 1928 are swapped in order hence the switch in the code

In [ ]:
import glob
ppt_files = glob.glob('/content/drive/MyDrive/SoilWat/Livneh_data/prec*.nc')
ppt_files = ppt_files[:66]

tmin_files = glob.glob('/content/drive/MyDrive/SoilWat/Livneh_data/tmin*.nc')
tmin_files = tmin_files[:66]

tmax_files = glob.glob('/content/drive/MyDrive/SoilWat/Livneh_data/tmax*.nc')
tmax_files = tmax_files[:66]
tmax_files[12], tmax_files[13] = tmax_files[13],tmax_files[12]


this method is great for pulling site by site but it is extremely slow

In [ ]:
import xarray as xr
for i in range(len(site)):
  id = site[i]
  lat_id = lat[i]
  lon_id = 360+lon[i]
  years = list(range(1915,1981))
  for i in range(len(ppt_files)):
    ppt=xr.open_dataset(ppt_files[i])
    ppt_ts=ppt.sel(lon=lon_id, lat=lat_id, method='nearest')
    tmax=xr.open_dataset(tmax_files[i])
    tmax_ts=tmax.sel(lon=lon_id,lat=lat_id,method='nearest')
    tmin=xr.open_dataset(tmin_files[i])
    tmin_ts=tmin.sel(lon=lon_id,lat=lat_id,method='nearest')

    ppt_ts=ppt_ts.to_array()
    tmax_ts=tmax_ts.to_array()
    tmin_ts=tmin_ts.to_array()

    ppt_df=pd.DataFrame(ppt_ts)
    tmax_df=pd.DataFrame(tmax_ts)
    tmin_df=pd.DataFrame(tmin_ts)

    year=[years[i]] * len(ppt_df.columns)

    new = pd.DataFrame({'ppt':ppt_df.iloc[0],'tmax':tmax_df.iloc[0],'tmin':tmin_df.iloc[0],
                      'yday':(ppt_df.columns+1),'year':year})
    if i == 0:
      final_df = new
    else:
      final_df = pd.concat([final_df,new])
  final_df.to_csv(f"/content/drive/MyDrive/Thesis/Chapter2_simulations/livneh/{id}.csv")

In [ ]:
year = range(1915,1981)

fast method! results in yearly csv's with a row for each site

In [ ]:
import xarray as xr

for i in range(len(ppt_files)):
  ppt=xr.open_dataset(ppt_files[i])
  big_ppt = ppt.sel(lon=xr.DataArray(lon), lat=xr.DataArray(lat), method='nearest')
  big_ppt=big_ppt.to_array()
  big_ppt = big_ppt.reindex(space=['dim_0'])
  big_ppt_df = pd.DataFrame(big_ppt[0,:,:])
  t_ppt_df = big_ppt_df.transpose()
  t_ppt_df['site'] = site
  t_ppt_df['year'] = year[i]
  t_ppt_df.to_csv(f"/content/drive/MyDrive/Thesis/Chapter2_simulations/livneh/ppt_{year[i]}.csv")
  tmax=xr.open_dataset(tmax_files[i])
  big_tmax = tmax.sel(lon=xr.DataArray(lon), lat=xr.DataArray(lat), method='nearest')
  big_tmax=big_tmax.to_array()
  big_tmax = big_tmax.reindex(space=['dim_0'])
  big_tmax_df = pd.DataFrame(big_tmax[0,:,:])
  t_tmax_df = big_tmax_df.transpose()
  t_tmax_df['site'] = site
  t_tmax_df['year'] = year[i]
  t_tmax_df.to_csv(f"/content/drive/MyDrive/Thesis/Chapter2_simulations/livneh/tmax_{year[i]}.csv")
  tmin=xr.open_dataset(tmin_files[i])
  big_tmin = tmin.sel(lon=xr.DataArray(lon), lat=xr.DataArray(lat), method='nearest')
  big_tmin=big_tmin.to_array()
  big_tmin = big_tmin.reindex(space=['dim_0'])
  big_tmin_df = pd.DataFrame(big_tmin[0,:,:])
  t_tmin_df = big_tmin_df.transpose()
  t_tmin_df['site'] = site
  t_tmin_df['year'] = year[i]
  t_tmin_df.to_csv(f"/content/drive/MyDrive/Thesis/Chapter2_simulations/livneh/tmin_{year[i]}.csv")

# Part 2

Load in the yearly csv's created above

In [ ]:
import glob
tmin_files = glob.glob('/content/drive/MyDrive/Thesis/Chapter2_simulations/livneh/tmin_*.csv')
tmax_files = glob.glob('/content/drive/MyDrive/Thesis/Chapter2_simulations/livneh/tmax_*.csv')
ppt_files = glob.glob('/content/drive/MyDrive/Thesis/Chapter2_simulations/livneh/ppt_*.csv')

In [ ]:
import pandas as pd
test = pd.read_csv(tmin_files[1])

In [ ]:
df = pd.DataFrame(test.iloc[:,1:-2].values.ravel(), columns=['Column'])

In [ ]:
df['site'] = [ele for ele in test['site'] for i in range(len(test.axes[1])-3)]
df['year'] = 1916

This loop creates a single dataframe, where each day (row) for all years has ppt, tmin, tmax for each site (898)

In [ ]:
for i in range(len(tmin_files)):
  tmin = pd.read_csv(tmin_files[i])
  tmin_long = pd.DataFrame(tmin.iloc[:,1:-2].values.ravel(), columns = ['Tmin_C'])
  tmin_long['site'] = [ele for ele in tmin['site'] for j in range(len(tmin.axes[1])-3)]
  tmin_long['year'] = tmin['year'][1]
  tmin_long['doy'] = list((range(1,(len(tmin.axes[1])-2))))*len(tmin.index)
  if i == 0:
    tmin_big = tmin_long
  else:
    tmin_big = pd.concat([tmin_big,tmin_long])
  ##############################################
  tmax = pd.read_csv(tmax_files[i])
  tmax_long = pd.DataFrame(tmax.iloc[:,1:-2].values.ravel(), columns = ['Tmax_C'])
  tmax_long['site'] = [ele for ele in tmax['site'] for j in range(len(tmax.axes[1])-3)]
  tmax_long['year'] = tmax['year'][1]
  tmax_long['doy'] = list((range(1,(len(tmax.axes[1])-2))))*len(tmax.index)
  if i == 0:
    tmax_big = tmax_long
  else:
    tmax_big = pd.concat([tmax_big,tmax_long])
  ##############################################
  ppt = pd.read_csv(ppt_files[i])
  ppt_long = pd.DataFrame(ppt.iloc[:,1:-2].values.ravel(), columns = ['PPT'])
  ppt_long['site'] = [ele for ele in ppt['site'] for j in range(len(ppt.axes[1])-3)]
  ppt_long['year'] = ppt['year'][1]
  ppt_long['doy'] = list((range(1,(len(ppt.axes[1])-2))))*len(ppt.index)
  if i == 0:
    ppt_big = ppt_long
  else:
    ppt_big = pd.concat([ppt_big,ppt_long])

In [ ]:
full = pd.merge(ppt_big, tmax_big,  on=['site','year','doy'])
full = pd.merge(full, tmin_big,  on=['site','year','doy'])
full.to_csv("/content/drive/MyDrive/Thesis/Chapter2_simulations/livneh/all_plots_climate.csv")